## **PASSNYC Dataset Training:**

We might need to remove all NaN values/DNE values in our dataset, right now I'm making them equal to 0, but that is probably messing with the neural network. Does that mean we need to remove it for every feature we use?

Latitude and Longitude doesn't necessarily need to be included as the features, becuase the School District information is already being passed in.

In [1]:
import pandas as pd
import sys
import tensorflow as tf
import numpy as np
import keras
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import *
from keras.models import load_model

Using TensorFlow backend.


Data loaded from the two datasets provided and the NYT SHSAT article

In [0]:
schoolExplorer = pd.read_csv("kaggle-data/2016 School Explorer.csv", encoding = 'utf-8')
registrationTesters = pd.read_csv("kaggle-data/D5 SHSAT Registrations and Testers.csv", encoding= 'utf-8')
nytdf = pd.read_csv("kaggle-data/nytdf.csv", encoding = 'utf-8')
highschoolDirectory = pd.read_csv("kaggle-data/2017-doe-high-school-directory.csv",encoding = 'utf-8')
middleschoolDirectory = pd.read_csv("kaggle-data/2018_DOE_Middle_School_Directory.csv",encoding = 'utf-8')


highschoolDirectory = highschoolDirectory.set_index('dbn')
middleschoolDirectory = middleschoolDirectory.set_index('schooldbn')


schoolExplorer = schoolExplorer.set_index('Location Code')
nytdf = nytdf.set_index('DBN')

explorer_nyt = schoolExplorer.join(nytdf)
nytCombined = nytdf.join(schoolExplorer)

nytCombined.to_csv("kaggle-data/combined.csv", index=False)
testErrorArray = []

In [0]:
nytCombined.shape

(589, 167)

The rationale behind creating a neural network with the NumofSHSATTestTakers is to determine the most probable schools which would have more test takers and then predict other schools' probable students. Following which we can rank each school depending on how diverse they are

In [0]:
# Filter the dataset to remove any schools without any SHSAT Test takers

features = ['TestTakersPercentage','ELA8thGrade4s','Math8thGrade4s','Percent of Students Chronically Absent','Average Math Proficiency','Economic Need Index','District','Average ELA Proficiency']

nytCombinedFiltered = nytCombined.query("NumSHSATTestTakers !=0")

Math8thGrade4s = pd.Series(nytCombinedFiltered[ 'Grade 8 Math 4s - All Students']/(nytCombinedFiltered[ 'Grade 8 Math - All Students Tested']*1.0))
ELA8thGrade4s = pd.Series(nytCombinedFiltered[ 'Grade 8 ELA 4s - All Students']/(nytCombinedFiltered[ 'Grade 8 ELA - All Students Tested']*1.0))
TestTakersPercentage = pd.Series(nytCombinedFiltered['NumSHSATTestTakers']/(nytCombinedFiltered[ 'Grade 8 Math - All Students Tested']*1.0))

nytCombinedFiltered = nytCombinedFiltered.assign(Math8thGrade4s = Math8thGrade4s)
nytCombinedFiltered = nytCombinedFiltered.assign(ELA8thGrade4s = ELA8thGrade4s)
nytCombinedFiltered = nytCombinedFiltered.assign(TestTakersPercentage = TestTakersPercentage)


def removePercent(df,columnName):
    df[columnName] = df[columnName].str.rstrip('%').astype('float') / 100.0
    features.append(columnName)
    return

removePercent(nytCombinedFiltered,'Student Attendance Rate')
removePercent(nytCombinedFiltered,'Percent of Students Chronically Absent')

nytCombinedFiltered.fillna(0,inplace = True)
nytCombinedFiltered = nytCombinedFiltered.replace([np.inf, -np.inf], np.nan)
nytCombinedFiltered = nytCombinedFiltered.dropna(subset=['TestTakersPercentage'])
# nytCombinedFiltered.fillna(1,inplace = True)

In [0]:
numFeatures = len(features)
print(numFeatures)

10


In [0]:
# nytCombinedFiltered['Student Attendance Rate']

In [0]:
# We split the Combined and filtered dataset to training and testing datasets, 60 each
datasetSplit = 150

nytCombinedFilteredTraining = nytCombinedFiltered[0:datasetSplit]
nytCombinedFilteredTesting = nytCombinedFiltered[datasetSplit:]

# Only pick out the columns that we want, ie: the ones with numeric values, we can still access the exact school name cause of the index

nytTraining = nytCombinedFilteredTraining.filter(features)
nytTesting = nytCombinedFilteredTesting.filter(features)


In [0]:
# list(nytTesting)



In [0]:
# Scale the inputs and outputs to a suitable small range
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale both the training inputs and outputs
scaled_training = scaler.fit_transform(nytTraining)
scaled_testing = scaler.transform(nytTesting)

scaled_training_df = pd.DataFrame(scaled_training, columns= nytTraining.columns.values)
scaled_testing_df = pd.DataFrame(scaled_testing, columns=nytTesting.columns.values)

scaled_training_df.to_csv("kaggle-data/training_scaled.csv", index=False)
scaled_testing_df.to_csv("kaggle-data/testing_scaled.csv", index=False)

print("Note: Number of SHSAT Taker values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[0], scaler.min_[0]))

Note: Number of SHSAT Taker values were scaled by multiplying by 0.1331344397 and adding -0.006340


In [0]:
#This is the training of the model

#This didnt work trying not the scaled values

X = scaled_training_df.drop('TestTakersPercentage', axis=1).values

# Using the output as a percentage

Y = scaled_training_df[['TestTakersPercentage']].values


# X = nytTraining.drop('NumSHSATTestTakers', axis=1).values
# Y = nytTraining[['NumSHSATTestTakers']].values


In [0]:
# Define the model
model = Sequential()
activationFuntion = 'relu'
model.add(Dense(50, input_dim=numFeatures-1, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))
# model.compile(loss="mean_squared_error", optimizer="adam")

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae','accuracy'])


#Train
model.fit(
    X,
    Y,
    epochs=50,
    shuffle=True,
    verbose=2
)

X_test = scaled_testing_df.drop('TestTakersPercentage', axis=1).values
Y_test = scaled_testing_df[['TestTakersPercentage']].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)

testErrorArray.append(test_error_rate)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))
print("Here: \n%s: %.5f%%" % (model.metrics_names[2], test_error_rate[2]))

model.save('savedModel.h5')
print("Model Saved to Folder")

Epoch 1/50
 - 1s - loss: 0.0138 - mean_absolute_error: 0.0621 - acc: 0.0067
Epoch 2/50
 - 0s - loss: 0.0105 - mean_absolute_error: 0.0580 - acc: 0.0067
Epoch 3/50
 - 0s - loss: 0.0096 - mean_absolute_error: 0.0498 - acc: 0.0067
Epoch 4/50
 - 0s - loss: 0.0089 - mean_absolute_error: 0.0432 - acc: 0.0067
Epoch 5/50
 - 0s - loss: 0.0082 - mean_absolute_error: 0.0470 - acc: 0.0067
Epoch 6/50
 - 0s - loss: 0.0076 - mean_absolute_error: 0.0410 - acc: 0.0067
Epoch 7/50
 - 0s - loss: 0.0072 - mean_absolute_error: 0.0443 - acc: 0.0067
Epoch 8/50
 - 0s - loss: 0.0067 - mean_absolute_error: 0.0399 - acc: 0.0067
Epoch 9/50
 - 0s - loss: 0.0065 - mean_absolute_error: 0.0406 - acc: 0.0067
Epoch 10/50
 - 0s - loss: 0.0063 - mean_absolute_error: 0.0422 - acc: 0.0067
Epoch 11/50
 - 0s - loss: 0.0062 - mean_absolute_error: 0.0414 - acc: 0.0067
Epoch 12/50
 - 0s - loss: 0.0067 - mean_absolute_error: 0.0412 - acc: 0.0067
Epoch 13/50
 - 0s - loss: 0.0065 - mean_absolute_error: 0.0520 - acc: 0.0067
Epoch 14

In [0]:
#Loading the saved model and then using it:

model = load_model('savedModel.h5')

# prediction = model.predict(X)

In [0]:
X = pd.read_csv('kaggle-data/training_scaled.csv')


X.head()

X = X.loc[0:10]

before = X[0:20]['TestTakersPercentage']
X = X.drop('TestTakersPercentage',axis = 1)
prediction = model.predict(X)
# prediction = prediction + scaler.min_[0]
prediction = prediction / scaler.scale_[0]


print(before,prediction)



0     0.092235
1     0.115882
2     1.000000
3     0.226646
4     0.510349
5     0.102989
6     0.123342
7     0.134662
8     0.087131
9     0.075038
10    0.079187
Name: TestTakersPercentage, dtype: float64 [[0.8120292 ]
 [0.77867067]
 [4.939108  ]
 [1.6150904 ]
 [4.0077496 ]
 [0.69094396]
 [0.8725062 ]
 [1.1068518 ]
 [0.51014334]
 [0.7604499 ]
 [0.5523411 ]]
